# 2D solution arrays and right-hand-side functions

While our little Euler code in the first notebook was useful for understanding the basic ideas behind solving ODEs numerically, in practice one uses a library.
In the next notebook you will start using the SciPy ODE solver [odeint](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html). It will allow you to solve a variety of ODE and it will take care of the details for you. You simply need to learn to use it correctly.

There are two things that you will find useful to understand first, so we cover these in this notebook: 

- The solution array for a system of ODEs naturally needs to be a 2D array: one dimension corresponding to time and the other dimension corresponding to the dependent variables in the system. We will always call the solution array `y_sol` in these notebooks, but any other name could be used.

- We want to specify the right-hand side functions of our ODEs as Python functions. Given that we write our system of ODEs as

$$
\dot y = f(y,t), \label{eq:ODE1}\tag{1}
$$

it is very natural to want to express the mathematical RHS function $f(y,t)$ as a Python function. You may have already thought of this. 

---

## 2D solution arrays

Even though SciPy will take care of everything for you, it is important to have a clear understanding of the solution arrays returned by the routine  <span style="color: blue;">odeint</span>. Moreover, the material in this section will reinforce your general understanding of 
Python and arrays.


In our Python code below we will use the variable `Neqs` to denote the number of equations in our system, that is $y \in \mathbb R^{\text Neqs}$. When you use the SciPy library you will not need this variable. 

In [1]:
# import libraries

import numpy as np
import matplotlib.pyplot as plt

---
The cell below sets a number of time steps and sets an initial condition `y0` to be an array, in this case with two values. From the initial condition, the code computes the number of dependent variables, `Neqs`, as the length of `y0`. We assume that you have given the correct number of initial conditions for the system.

**Exercise:**  At the bottom of the cell, insert code to allocate `y_sol` as a 2D array of zeros, with first dimension equal to the number of time grid points and the second equal to the number of equations. Print the shape of `y_sol`. (Skim down to the code cell `Euler time-stepping ...` if you have trouble.) 

In [6]:
# Set number of time steps
Nsteps = 5

# Set initial condition
y0 = np.array([2, 4])

# Compute number of dependent variables from y0
Neqs = len(y0)

y_sol = np.zeros(shape=(6,2))
print(y_sol.shape)

(6, 2)


The shape should be `(6,2)`. The 6 is for the `Nsteps+1` time points and the 2 is for the number of equations `Neqs`. `np.zeros` is one of the functions you need to remember without looking up each time. You need to put two sets of parentheses if you want a 2D array. As always, we print shape with `print(y_sol.shape)`.

---

The time-stepping loop in our first ODE solver used `y[n+1] = y[n] + dt * ydot` to iteratively update the solution array. We need to generalise this to the case of `Neqs`. There is no exercise here except to run the cell below and understand it. Here we consider the trivial system of ODEs with $f(y) = y$,  so `ydot` will just be `y_sol` at time step `n`.

Observe the use of slicing -- we update all columns, that is all components, simultaneously using the colon notation. 

We also include a print statements so that you can see how the solution array is filled in with each iteration. (Note, if you want to run the cell more than once, you will need to re-run the cells above to reset `y_sol`.)

In [7]:
# set an artificial time step
dt = 0.2

# initialise the solution array with the initial condition
y_sol[0,:] = y0[:]
print(y_sol, "\n")

# time-stepping loop
for n in range(Nsteps):
    ydot = y_sol[n,:]
    y_sol[n+1,:] = y_sol[n,:] + dt * ydot
    print(y_sol, "\n")

[[2. 4.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]] 

[[2.  4. ]
 [2.4 4.8]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]] 

[[2.   4.  ]
 [2.4  4.8 ]
 [2.88 5.76]
 [0.   0.  ]
 [0.   0.  ]
 [0.   0.  ]] 

[[2.    4.   ]
 [2.4   4.8  ]
 [2.88  5.76 ]
 [3.456 6.912]
 [0.    0.   ]
 [0.    0.   ]] 

[[2.     4.    ]
 [2.4    4.8   ]
 [2.88   5.76  ]
 [3.456  6.912 ]
 [4.1472 8.2944]
 [0.     0.    ]] 

[[2.      4.     ]
 [2.4     4.8    ]
 [2.88    5.76   ]
 [3.456   6.912  ]
 [4.1472  8.2944 ]
 [4.97664 9.95328]] 



The SciPy library is going to take care of all the details of the time stepping for you. The point of the above example is to illustrate how the 2D solution array `y_sol` will be filled in as the solution is computed. Also, as a general Python skill, you really need to be able to read and use slicing.

---

The cell below combines the cells above into a time-stepping code that allows for flexibility in the number of equations. 

In [8]:
# Euler time-stepping of the systems of ODEs: dot y = y.

# Set problem parameters
Nsteps = 5
tf = 1
y0 = np.array([2, 4])

# --- what appears below should work for any, Nsteps, tf and y0 --- #

# Compute number of dependent variables from y0
Neqs = len(y0)

# Allocate t and y arrays
t, dt = np.linspace(0, tf, Nsteps+1, retstep=True)
y_sol = np.zeros((Nsteps+1, Neqs))

# Initialise the solution array
y_sol[0,:] = y0[:]

# Euler time-stepping loop (for the identity function f(t,y) = y)
for n in range(Nsteps):
    ydot = y_sol[n,:]
    y_sol[n+1,:] = y_sol[n,:] + dt * ydot

# print the final solution
print(y_sol)

[[2.      4.     ]
 [2.4     4.8    ]
 [2.88    5.76   ]
 [3.456   6.912  ]
 [4.1472  8.2944 ]
 [4.97664 9.95328]]


**Exercise:** Add a third value to the initial condition array `y0`. Maybe pick a negative one for variety. Rerun the cell. If you feel energetic, try other initial conditions and `Nsteps`, and observe the corresponding shape of the output. When you have finished, set the number of initial conditions back to 2 and `Nsteps` back to 5 and make sure the output is a 6 x 2 matrix. 

To summarise: 

- In the 2D solution array `y_sol`, rows correspond to different times and columns correspond to different components of the solution. 

---
### Extracting component solutions

After obtaining the solution array `y_sol`, we generally want to separate and plot different components. Often these components will correspond to variable with meaningful names. Suppose we want to extract the first component of `y_sol` into an array `S` and the second component into an array `I`. We again use slicing:

In [ ]:
S = y_sol[:,0]
I = y_sol[:,1]
print(S)
print(I)

We now have two arrays `S` and `I` whose index corresponds to the time step. Essentially we have `S` and `I` as functions of time and can do what we want with them. 

---

## RHS functions

We want a Python function that corresponds to the right-hand side of our systems of ODEs. 
It is best to just show an example. Consider the system of ODEs for a simple harmonic oscillator

$$
\begin{align}
\dot y_1 & = y_2  \\
\dot y_2 & = -y_1  
\end{align}
$$

The solution is (un-damped) simple harmonic motion with frequency 1, since the eigenvalues of $\begin{bmatrix} 0 & 1 \\ -1 & 0 \end{bmatrix}$ are $\pm i$.

We can write the RHS of this system as a Python function:

In [9]:
def harmonic(y, t):
    # returns RHS of simple harmonic oscillator ODE

    y1, y2 = y
    
    y1dot = y2
    y2dot = -y1

    return y1dot, y2dot

We have given the function a representative name. The argument `y` is a 1D array. It is one row of `y_sol` and corresponds to the solution at the "current time". The argument `t` is not used since this system is autonomous. 

Since the system of ODEs is written mathematically in terms of $y_1$ and $y_2$, we will use these names in our function. The line `y1, y2 = y` copies the elements of `y` to variables `y1` and `y2`. We will always use variable names in our functions that correspond as much as possible to variable names in the mathematical statement of the ODEs. 

We then set variables `y1dot` and `y2dot` according to the ODEs and return them. 

You might be wondering why we did not use any array manipulations in this function. You could, and it would not be wrong. However, we highly recommend you follow the model shown above. It will lead to far fewer errors. Moreover, it is common for the variables in systems of ODEs to have names other than $y_1$, $y_2$ etc. For example they might be $S$ and $I$.  For clarity, and to avoid mistakes, we will use these names as much as possible when writing our RHS functions. We will always follow the above example.  


---
# Exercise

1. Consider the damped harmonic oscillator ODEs

$$
\begin{align}
\dot y_1 & = y_2  \\
\dot y_2 & = - y_1 - \mu y_2,
\end{align}
$$

where $\mu \ge 0$ is a parameter. For $\mu = 0$ these equations become the un-damped harmonic oscillator.

Insert a code cell below. Using the function `harmonic` above as an example, define a function `damped` that computes the RHS of these equations and returns `y1dot` and `y2dot`. In the function you will need a variable `mu`. Set it to 0.25. 


---

2. Consider the system of ODEs corresponding to the SI model

$$
\dot S  = - \frac{\beta S I}{N}, \quad 
\dot I  = \frac{\beta S I}{N} 
$$

where $N = S+I$ and $\beta$ is a parameter. This exercise is for practice turning systems of ODEs into Python function. Later we will present a case study of the SIR model for you to understand more details. 

Insert a code cell below. Define a function `SI` that computes the RHS of these equations. It should extract variables `S` and `I` from the input argument `y`  and return `Sdot` and `Idot`. In the function you will need a variable `beta`. Set it to 0.1. 

---
# Answers
---

Expand cells (click on left margin) to see answers and any comments.

Q1 answer

In [ ]:
def damped(y, t):
    # returns RHS of damped harmonic oscillator ODE
    y1, y2 = y
    
    mu = 0.25 
    
    y1dot = y2
    y2dot = -y1 - mu * y2

    return y1dot, y2dot

Q2 answer

In [ ]:
def SI(y, t):
    # returns RHS of the SI model
    S, I = y

    beta = 0.1
    
    N = S + I 
    Sdot = -beta * S * I / N
    Idot = beta * S * I / N 
    
    return Sdot, Idot